In [1]:
##import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
from sklearn.utils import shuffle
import tqdm
from keras.models import load_model
import emoji
import re

Using TensorFlow backend.


In [2]:
def label_to_emoji(b):
    return emoji.emojize(emoji_dictionary[str(b)], use_aliases=True)

In [3]:
emoji_dictionary = {"0": ":sob:",    
                    "1": ":pensive:",
                    "2": ":smile:",
                    "3": ":innocent:"}

In [4]:
def user(c):
    c = np.asarray([[c]])
    corpus = []
    frame = re.sub('[^a-zA-Z]', ' ', c[0][0])
    frame = frame.lower()
    #frame = frame.split()
    #frame = ' '.join(c)
    corpus.append(frame)
    c_new = np.asarray(corpus)
    c_new = np.reshape(c_new, (len(corpus),1))
    indices = sentence_to_indices(c_new, word_to_index, max_len=40)
    a = model.predict(indices)

    if a<=0.3:
        b=0
        print("Highly Negative: ", label_to_emoji(b))
    elif a<=0.5:
        b=1
        print("Negative: ", label_to_emoji(b))
    elif a>=0.8:
        b=2
        print("Highly positive: ", label_to_emoji(b))
    else:
        b=3
        print("Positive: ", label_to_emoji(b))
    

In [5]:
def glove_vec(glove_file):
    with open (glove_file,'r') as f:
        words = set()
        words_to_vec = {}
        for line in f:
            line = line.strip().split()
            current_word = line[0]
            words.add(current_word)
            words_to_vec[current_word] = np.array(line[1:],dtype=np.float64)
            
        i = 1
        word_to_index = {}
        index_to_word = {}
        for w in sorted(words):
            word_to_index[w] = i
            index_to_word[i] = w
            i = i+1
            
    return word_to_index, index_to_word, words_to_vec

In [6]:
word_to_index, index_to_word, words_to_vec = glove_vec('../Highbrowsent/glove.6B/glove.6B.100d.txt')

In [7]:
from keras.models import Model,load_model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras import regularizers

In [20]:
def sentence_to_indices(X,word_to_index,max_len):
    m = X.shape[0]
    X_indices = np.zeros((m,max_len))
    for i in range(m):
        j = 0
        sentence_word = [word for word in X[i][0].split()]
        sentence_word = [word.lower() for word in X[i][0].split()]
        for w in sentence_word:
            if w in word_to_index.keys():
                X_indices[i, j] = word_to_index[w]
            else:
                X_indices[i, j] = word_to_index['unk']
            j =j + 1
            if j == maxLen:
                break
            
    return X_indices

In [9]:
maxLen = 40

In [10]:
model = load_model("../Highbrowsent/highbrowsent_f_model.h5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [11]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 40)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 40, 100)           40000100  
_________________________________________________________________
lstm_2 (LSTM)                (None, 40, 128)           117248    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 40,249,061
Trainable params: 248,961
Non-trainable params: 40,000,100
_________________________________________

In [21]:
user('my name is anurag')

Highly positive:  😄


In [22]:
user('The story is unpredictable')

Negative:  😔


In [23]:
user('The steering wheel is unpredictable.')

Highly Negative:  😭


In [24]:
user("I am happy")

Highly positive:  😄


In [25]:
user("I am not happy")

Highly Negative:  😭


In [26]:
user('This phone has an awesome battery backup of zero hours ')

Positive:  😇


In [27]:
user('he is too boring person but i like company with him')

Positive:  😇


In [28]:
user('i think that it will be worst seminar ever but it is very entertaining')

Highly positive:  😄


In [29]:
user("i am not happy")

Highly Negative:  😭
